# 🏡 Site Energy UI Prediction 

<img src = "https://www.clearias.com/up/energy-efficiency.jpg">

#### What the project is about?

### Section 1 : Setting Up The Project

In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import catboost as cb
import lightgbm as lgb
import matplotlib.pyplot as plt

from copy import deepcopy
from collections import defaultdict

from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import RandomizedSearchCV, train_test_split, LeaveOneGroupOut

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

### Section 2 : Helper Functions

In [2]:
def read_data(train_path, test_path):
    """Read in train and test data for a kaggle competition.

    Parameters
    ----------
    train_path : str
        The path to the training data.
    test_path : str
        The path to the test data

    Returns
    -------
    train_df, test_df : pandas DataFrames
        The train and test datasets.
    """
    train_df = pd.read_csv("train.csv")
    test_df = pd.read_csv("test.csv")
    return train_df, test_df

In [3]:
def get_duplicates(df, drop_cols=None):
    """Determine and return the duplicated values in a dataframe.

    Parameters
    ----------
    df : pandas DataFrame
        The dataframe to check
    drop_cols : str or list of str, optional
        The columns to drop before returning duplicates.

    Returns
    -------
    pandas DataFrame
        A dataframe containing the rows with duplicated values.
    """
    if drop_cols is not None:
        return df[df.drop(columns=drop_cols).duplicated()]
    else:
        return df[df.duplicated()]


In [4]:
def remove_duplicates(df, drop_cols=None):
    """Removes the duplicated values in a dataframe.

    Parameters
    ----------
    df : pandas DataFrame
        The dataframe to check
    drop_cols : str or list of str, optional
        The columns to drop before removing duplicates.
    Returns
    -------
    pandas DataFrame
        A dataframe without duplicated.
    """
    df_clean = df.copy()

    if drop_cols is not None:
        df_clean = df_clean[~df_clean.drop(columns=drop_cols).duplicated()]
    else:
        df_clean = df_clean[~df_clean.duplicated()]
    
    return df_clean.reset_index(drop=True)


In [5]:
def split_building_data(df, groups):
    """Splits the WiDS 2022 dataset based on groups of facility types.

    Parameters
    ----------
    df : pandas DataFrame
        The WiDS train or test dataset.
    groups : dict (key: str, value: set)
        The dictionary of facility types.  The key should be the desired group name.
        The value should be a set containing the facility types in a given group.

    Returns
    -------
    dfs : dict
        A dictionary of pandas DataFrames, one for each group.
    """
    dfs = {}

    for name, group in groups.items():
        group_df = df.query("facility_type in @group")
        dfs[name] = group_df.reset_index(drop=True)

    return dfs


In [6]:
# Scope to refactor
def get_manual_facility_groups():
    """Returns the manual facility groups used in my final WiDS solution.

    Returns
    -------
    The dictionary of facility types.
        The key is group name (first word in facility type).
        The value is a set containing the facility types in a given group.
    """

    facility_groups = {
        "2to4_5plus_Mixed": {
            "2to4_Unit_Building",
            "5plus_Unit_Building",
            "Mixed_Use_Predominantly_Residential",
        },
        "Commercial_Education_Mixed_Industrial_Parking": {
            "Education_College_or_university",
            "Education_Other_classroom",
            "Education_Preschool_or_daycare",
            "Education_Uncategorized",
            "Commercial_Other",
            "Commercial_Unknown",
            "Mixed_Use_Commercial_and_Residential",
            "Mixed_Use_Predominantly_Commercial",
            "Industrial",
            "Parking_Garage",
        },
        "Food_Grocery": {
            "Food_Sales",
            "Food_Service_Other",
            "Food_Service_Restaurant_or_cafeteria",
            "Food_Service_Uncategorized",
            "Grocery_store_or_food_market",
        },
        "Health": {
            "Health_Care_Inpatient",
            "Health_Care_Outpatient_Clinic",
            "Health_Care_Outpatient_Uncategorized",
            "Health_Care_Uncategorized",
        },
        "Laboratory_Data": {"Laboratory", "Data_Center"},
        "Lodging": {
            "Lodging_Dormitory_or_fraternity_sorority",
            "Lodging_Hotel",
            "Lodging_Other",
            "Lodging_Uncategorized",
        },
        "Multifamily": {"Multifamily_Uncategorized"},
        "Office_Nursing": {
            "Office_Bank_or_other_financial",
            "Office_Medical_non_diagnostic",
            "Office_Mixed_use",
            "Office_Uncategorized",
            "Nursing_Home",
        },
        "Public": {
            "Public_Assembly_Drama_theater",
            "Public_Assembly_Entertainment_culture",
            "Public_Assembly_Library",
            "Public_Assembly_Movie_Theater",
            "Public_Assembly_Other",
            "Public_Assembly_Recreation",
            "Public_Assembly_Social_meeting",
            "Public_Assembly_Stadium",
            "Public_Assembly_Uncategorized",
            "Public_Safety_Courthouse",
            "Public_Safety_Fire_or_police_station",
            "Public_Safety_Penitentiary",
            "Public_Safety_Uncategorized",
        },
        "Religious": {"Religious_worship"},
        "Retail": {
            "Retail_Enclosed_mall",
            "Retail_Strip_shopping_mall",
            "Retail_Uncategorized",
            "Retail_Vehicle_dealership_showroom",
        },
        "Warehouse_Service": {
            "Warehouse_Distribution_or_Shipping_center",
            "Warehouse_Nonrefrigerated",
            "Warehouse_Refrigerated",
            "Warehouse_Selfstorage",
            "Warehouse_Uncategorized",
            "Service_Drycleaning_or_Laundry",
            "Service_Uncategorized",
            "Service_Vehicle_service_repair_shop",
        },
    }

    return facility_groups


In [7]:
def count_missing(df):
    """Counts the missing data in a dataframe.

    Parameters
    ----------
    df : pandas DataFrame
        The dataframe to count the missing data in.

    Returns
    -------
    pandas DataFrame
        A summary of missing data (counts and %)
    """
    missing_df = pd.DataFrame(
        df.isna().sum().sort_values(ascending=False), columns=["count"]
    )
    missing_df["percent"] = missing_df["count"] / df.shape[0]
    return missing_df.query("count != 0")

In [8]:
def iterative_impute(train_df, test_df, model, ct, target, feat_names, seed):
    """Imputes missing data into train and test datasets with a ML model of choice.

    Parameters
    ----------
    train_df : pandas DataFrame
        The training dataset
    test_df : pandas DataFrame
        The test dataset
    model : sklearn estimator
        The machine learning model to use for imputation
    ct : sklearn ColumnTransformer
        The column transformer to perform on the dataset
    target : str
        The target variable (removed before imputation)
    feat_names : list
        Names of features to append to OHE features from column transformer.
    seed : int
        The random seed for imputation.

    Returns
    -------
    train_imp, test_imp
        Train and test datasets with imputed values.
    """
    train_imp = ct.fit_transform(train_df.drop(columns=[target]))
    test_imp = ct.transform(test_df)

    imputer = IterativeImputer(estimator=model, random_state=seed)

    cols = (
        ct.named_transformers_["onehotencoder"].get_feature_names().tolist()
        + feat_names
    )

    train_imp = pd.DataFrame(imputer.fit_transform(train_imp), columns=cols)
    test_imp = pd.DataFrame(imputer.transform(test_imp), columns=cols)

    return train_imp, test_imp

In [9]:
# the columns that are imputed in the above code block are put back into the original dataset here
def replace_columns(df, df_imp, columns):
    """Replace columns in a dataframe with columns from another.

    Note: Meant for use with imputed datasets for WiDS 2022.

    Parameters
    ----------
    df : pandas DataFrame
        The original dataframe.
    df_imp : pandas DataFrame
        The imputed dataframe.
    columns : str or list of str
        The columns to replace between dataframes.

    Returns
    -------
    df_replaced : pandas DataFrame
        Dataframe with replaced columns.
        
    """
    df_replaced = df.copy()

    for col in columns:
        df_replaced[col] = df_imp[col]

    return df_replaced

In [10]:
# Scope for refactoring
def impute_and_replace(
    train_dfs, test_dfs, model, ct, target, feat_names, replace, seed
):
    """Iteratively impute multiple dataframes.

    Since I have 12 individual datasets based on the groups of facility_types, I am caling the imputer and replace functions
    on each of those datasets in this function
    """
    train_dfs_imp = {}
    test_dfs_imp = {}

    # iterative imputation
    for (name1, train_df), (name2, test_df) in zip(train_dfs.items(), test_dfs.items()):
        train_imp, test_imp = iterative_impute(
            train_df, test_df, model, ct, target, feat_names, seed
        )
        train_dfs_imp[name1] = train_imp
        test_dfs_imp[name1] = test_imp

    # replace train columns with missing values
    for (name1, df), (name2, imp_df) in zip(train_dfs.items(), train_dfs_imp.items()):
        train_dfs[name1] = replace_columns(df, imp_df, replace)

    # replace test columns with missing values
    for (name1, df), (name2, imp_df) in zip(test_dfs.items(), test_dfs_imp.items()):
        test_dfs[name1] = replace_columns(df, imp_df, replace)

    return train_dfs, test_dfs

In [11]:
def has_feature(df, feat):
    """Adds a boolean feature if a feature exists or now"""
    return df[feat].notna().astype(int)

# Avoids repeated use of groupbys in the notebook
def group_by_feature(train_df, test_df, group, agg_feat, transform, name):
    """Adds a new aggregated feature based on a categorical variable"""
    train_df_new = train_df.copy()
    test_df_new = test_df.copy()
    
    fill_values = train_df_new.groupby(group).aggregate(transform)[agg_feat].to_dict()
    
    train_df_new[name] = train_df_new[group].map(fill_values)
    test_df_new[name] = test_df_new[group].map(fill_values)
    
    return train_df_new, test_df_new

In [12]:
def bin_days_below_above(df):
    """Bins the days below and above features into 4 bins"""
    df_eng = df.copy()
    
    df_eng["freezing_days"] = df_eng['days_below_0F'] + df_eng['days_below_10F']
    df_eng["cold_days"] = df_eng['days_below_20F'] + df_eng['days_below_30F']
    df_eng["warm_days"] = df_eng['days_above_80F'] + df_eng['days_above_90F']
    df_eng["hot_days"] = df_eng['days_above_100F'] + df_eng['days_above_110F']
    
    return df_eng

In [13]:
def seasonal_temps(df, stat):
    """Add seasonal temperatures for a given statistic specified by user"
    """
    df_eng = df.copy()

    df_eng[f"winter_{stat}"] = (
        df_eng[f"december_{stat}"]
        + df_eng[f"january_{stat}"]
        + df_eng[f"february_{stat}"]
    ) / 3

    df_eng[f"spring_{stat}"] = (
        df_eng[f"march_{stat}"] + df_eng[f"april_{stat}"] + df_eng[f"june_{stat}"]
    ) / 3

    df_eng[f"summer_{stat}"] = (
        df_eng[f"june_{stat}"] + df_eng[f"july_{stat}"] + df_eng[f"august_{stat}"]
    ) / 3
    
    df_eng[f"autumn_{stat}"] = (
        df_eng[f"september_{stat}"]
        + df_eng[f"october_{stat}"]
        + df_eng[f"november_{stat}"]
    ) / 3

    return df_eng



In [14]:
def feature_engineer(train_df, test_df):
    train_df_eng = train_df.copy()
    test_df_eng = test_df.copy()

    # whether or not a building has a fog detector, This I have added because majority of buildings do not have a fog detector
    
    train_df_eng["has_fog_detector"] = has_feature(train_df_eng, "days_with_fog")
    test_df_eng["has_fog_detector"] = has_feature(test_df_eng, "days_with_fog")

    # whether or not a building has a wind detector
    train_df_eng["has_wind_detector"] = has_feature(train_df_eng, "max_wind_speed")
    test_df_eng["has_wind_detector"] = has_feature(test_df_eng, "max_wind_speed")
    
    # seasonal avg temps
    train_df_eng = seasonal_temps(train_df_eng, "avg_temp")
    test_df_eng = seasonal_temps(test_df_eng, "avg_temp")
    
    # aggregate features
    agg_feats = ["energy_star_rating", "floor_area", "ELEVATION"]

    for agg_feat in agg_feats:
        name = "mean_" + agg_feat
        train_df_eng, test_df_eng = group_by_feature(
            train_df_eng, test_df_eng, "facility_type", agg_feat, "mean", name
        )
    
    # whether or not energy star is better than mean for facility
    train_df_eng["e_star_better_than_mean"] = (
        train_df_eng["energy_star_rating"] > train_df_eng["mean_energy_star_rating"]
    ).astype(int)
    test_df_eng["e_star_better_than_mean"] = (
        test_df_eng["energy_star_rating"] > test_df_eng["mean_energy_star_rating"]
    ).astype(int)
    
    # total snow and rain
    train_df_eng["snow_rain_inches"] = (
        train_df_eng["snowfall_inches"] + train_df_eng["precipitation_inches"]
    )
    test_df_eng["snow_rain_inches"] = (
        test_df_eng["snowfall_inches"] + test_df_eng["precipitation_inches"]
    )

    # total degree days
    train_df_eng["degree_days"] = (
        train_df_eng["cooling_degree_days"] + train_df_eng["heating_degree_days"]
    )
    test_df_eng["degree_days"] = (
        test_df_eng["cooling_degree_days"] + test_df_eng["heating_degree_days"]
    )
    

    